# Removng Ancestors from the Phylogenetic Tree
Program to prune the phylogenetic tree file of all species to only those modern species that would be used for ASR. Looks at the nodes the ancestral sequences will occupy to keep a track of them. 

Milo Thordarson (anth2886@student.uu.se)

## Loading packages and files

In [3]:
from ete3 import Tree
from Bio import SeqIO
import pandas

# We need the SNP file, as it has the annotations of which sequences are Ancestral
tree_file = "treefile_pette"
snp_file = "snpAlignment_Spyrou2022_modern_ancient.fasta2"

## Creating list of ancient species

In [11]:
# First, read in the treefile as a Tree in ete3 and snp file as a list from a fasta
full_tree = Tree(tree_file)
snp_fasta = list(SeqIO.parse(open(snp_file), 'fasta'))

print(f'Number of leaves in tree file: {len(full_tree)}')
print(f'Number of leaves in snp file: {len(snp_fasta)}')

# Then root the tree using the outgroup
full_tree.set_outgroup("outgroup_Y.pseudo")

Number of leaves in tree file: 251
Number of leaves in snp file: 251


In [5]:
# Now get ids from snp file that are ancient, removing the last 8 characters that are the tag
ancient = [x.id[:-8] for x in snp_fasta if "Ancient" in x.id]
print(f'Number of ancient samples to start: {len(ancient)}')

# Get names of all species from the treefile
tree_names = list(full_tree.iter_leaf_names())

# For loop to go through and make sure that the ancient names from the snp file match those in the tree exactly
for i in range(len(ancient)):
    if ancient[i] not in tree_names:
        # Finding the name as the prefix matches
        name = next((name for name in tree_names if ancient[i] in name), None)
        if name is not None:
            ancient[i] = name
        # Well, the prefix matches, except for this known exception, so I am doing it manually
        elif ancient[i] == "COL1":
            ancient[i] = "COLC1-COLC2a_COLC2b"

# Double check that we are left with a list of the same length of ancient samples, and that all the names are found in the treefile
print(f'Number of ancient samples: {len(ancient)}, all ancient names found in tree names: {all(x in tree_names for x in ancient)}')

Number of ancient samples to start: 47
Number of ancient samples: 47, all ancient names found in tree names: True


## Annotation of Ancient Samples

In [6]:
# List to keep a track of the branch length of each ancient sample
distances = []

# Add distances to list
for node_name in ancient:
    distances.append(full_tree.search_nodes(name=node_name)[0].dist)

# Create pandas dataframe to keep track of information about each ancient sample
d = {'name': ancient, 'dist': distances}
ancient_df = pandas.DataFrame(data = d)

# Displaying it sorted by the distance, so we can see which ancient sequences are closest to being the real ancestral sequence. 
ancient_df.sort_values(by=['dist'], ascending=True)

,name,dist
42,OBS124,0.000002
0,BSK001-003.A0101.A0102.A0103-malt,0.000002
39,London_EastSmithfield_8124_8291_11972,0.000002
36,STN019.A0101,0.000002
35,STN008.A0101,0.000002
34,STN021.A0101,0.000002
33,STN020.A0101,0.000002
32,STN014.A0101,0.000002
31,STN013.A0101,0.000002
30,STN007.A0101,0.000002


In [7]:
# This is just a small code to name all of the interal nodes something informative. 

count = 0

for node in full_tree.traverse():
  if not node.is_leaf():
    node.name = "intrnl" + str(count)
    count += 1

### Annotation for loop

The interpretation of the ASR sequences will be different if the ancient sequence we are looking at is a direct sister group to any taxa: that means that it is not an ancestor at a node, but along the evolution of the mdoern branch and therefore will not be reconstructed. But if the sister taxa is another ancient sequence, then ASR will reconstruct the ancestor to both those sequences and the modern, meaning that interpretation is harder. 

In [8]:
# Seting up a count for the number of times an issue is found and a list so they can be coded into the dataframe
ancient_problem_count = 0
ancient_sisters = []
sister_problem_count = 0
sisters = []

# Also keeps a track of the parent nodes for the dataframe
parents = []

for node_name in ancient:
    # Set up check for problems, and store parent names
    ancient_problem = False
    sister_problem = False  
    node = full_tree.search_nodes(name=node_name)[0]
    parent = node.up
    parents.append(parent.name)

    for child in parent.get_children():
        # Make sure you don't count the ancient sample we're already looking at
        if child.is_leaf() and child.name != node_name: 
            if child.name in ancient: 
                ancient_problem_count += 1
                ancient_problem = True
                ancient_sisters.append(child.name)
            else:
                sister_problem_count += 1
                sister_problem = True
                sisters.append(child.name)
    # If no problems were found, code that with a dash
    if ancient_problem == False: ancient_sisters.append("-")
    if sister_problem == False: sisters.append("-")

print(f'Number of ancient sister problems = {ancient_problem_count}')
print(f'Number of sister problems = {sister_problem_count}')
print(f'Total problematic ancient samples = {ancient_problem_count + sister_problem_count}')
print(f'Total useful ancient samples = {len(ancient) - (ancient_problem_count + sister_problem_count)}')

Number of ancient sister problems = 18
Number of sister problems = 0
Total problematic ancient samples = 18
Total useful ancient samples = 29


In [9]:
# Adding the information to the dataframe

ancient_df['parent'] = parents
ancient_df['anct_problem'] = ancient_sisters
ancient_df['sis_problem'] = sisters

ancient_df.sort_values(by=['dist'], ascending=True)

,name,dist,parent,anct_problem,sis_problem
42,OBS124,0.000002,intrnl246,OBS116,-
0,BSK001-003.A0101.A0102.A0103-malt,0.000002,intrnl69,-,-
39,London_EastSmithfield_8124_8291_11972,0.000002,intrnl126,-,-
36,STN019.A0101,0.000002,intrnl228,-,-
35,STN008.A0101,0.000002,intrnl220,-,-
34,STN021.A0101,0.000002,intrnl247,STN013.A0101,-
33,STN020.A0101,0.000002,intrnl243,-,-
32,STN014.A0101,0.000002,intrnl204,-,-
31,STN013.A0101,0.000002,intrnl247,STN021.A0101,-
30,STN007.A0101,0.000002,intrnl213,-,-


## Creating a pruned tree

In [49]:
# Prune the tree, agrument taken is species to retain
full_tree.prune([x for x in tree_names if x not in ancient], preserve_branch_length=True)
pruned_tree = full_tree
print(f'{len(pruned_tree)}')

211


In [50]:
# Then create a pruned tree file in newick format
pruned_tree.write(format=1, outfile="pruned_tree_outgroup_dist.treefile")